In [11]:
# Logistic Regression - Survey of women in 1974 by Redbook magazine, in which married women were asked about their participation in extramarital affairs. More information about the study is available in a 1978 paper from the Journal of Political Economy.
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
%matplotlib inline
dta = sm.datasets.fair.load_pandas().data

In [5]:
dta.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666


In [25]:
# Creating dataframes with an intercept column and dummy variables for occupation and occupation_husb
y, x = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
religious + educ + C(occupation) + C(occupation_husb)',dta, return_type="dataframe")
# Renaming the column names for better readability
x = x.rename(columns = {'C(occupation)[T.2.0]':'occ_2',
'C(occupation)[T.3.0]':'occ_3',
'C(occupation)[T.4.0]':'occ_4',
'C(occupation)[T.5.0]':'occ_5',
'C(occupation)[T.6.0]':'occ_6',
'C(occupation_husb)[T.2.0]':'occ_husb_2',
'C(occupation_husb)[T.3.0]':'occ_husb_3',
'C(occupation_husb)[T.4.0]':'occ_husb_4',
'C(occupation_husb)[T.5.0]':'occ_husb_5',
'C(occupation_husb)[T.6.0]':'occ_husb_6'})
# flatten y into a 1-D array
y = np.ravel(y)

In [37]:
# Evaluating our model by splitting into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [63]:
model.score(x_train,y_train)
# Getting almost 73% of accuracy in our train model
names = [i for i in list(x_train)]
# Analyzing the coefficents
train_coef = pd.DataFrame(list(zip(names,np.transpose(model.coef_))),columns=["features","coefficients"])
print(train_coef)
print("From the coefficients we could infer that increase in marriage rating and religiousness correspond to a decrease in the likelihood of having an affair")

         features             coefficients
0       Intercept     [1.4123499097864722]
1           occ_2    [0.30553545458838827]
2           occ_3     [0.5937898221274117]
3           occ_4    [0.31149019931559907]
4           occ_5     [0.9460879235696249]
5           occ_6     [1.0653566303329938]
6      occ_husb_2    [0.11851764693921081]
7      occ_husb_3    [0.24200844320063422]
8      occ_husb_4     [0.0806531374715437]
9      occ_husb_5    [0.04340524171611639]
10     occ_husb_6   [0.026564834566965952]
11  rate_marriage    [-0.7047861761597323]
12            age   [-0.05379518560053637]
13    yrs_married      [0.105003158701278]
14       children  [-0.011343388480559989]
15      religious   [-0.38184213480130524]
16           educ    [0.01265828509332857]
From the coefficients we could infer that increase in marriage rating and religiousness correspond to a decrease in the likelihood of having an affair


In [58]:
# Prediction set for test dataset
predicted = model.predict(x_test)
prob = model.predict_proba(x_test)
print(predicted)
print(prob)

[1. 0. 0. ... 0. 0. 0.]
[[0.35142488 0.64857512]
 [0.90952576 0.09047424]
 [0.72576603 0.27423397]
 ...
 [0.55736751 0.44263249]
 [0.81213933 0.18786067]
 [0.74729595 0.25270405]]


In [61]:
# Evaluation metrics
print(metrics.accuracy_score(y_test, predicted))
print(metrics.roc_auc_score(y_test, prob[:, 1]))
print(metrics.confusion_matrix(y_test, predicted))
print(metrics.classification_report(y_test, predicted))

0.7298429319371728
0.7459619860896347
[[1169  134]
 [ 382  225]]
             precision    recall  f1-score   support

        0.0       0.75      0.90      0.82      1303
        1.0       0.63      0.37      0.47       607

avg / total       0.71      0.73      0.71      1910



In [62]:
# Evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), x, y, scoring='accuracy', cv=10)
scores, scores.mean()
print("Hence our model has an accuracy score of 73%")

(array([0.72100313, 0.70219436, 0.73824451, 0.70597484, 0.70597484,
        0.72955975, 0.7327044 , 0.70440252, 0.75157233, 0.75      ]),
 0.7241630685514876)